In [1]:
import pandas as pd
import mysql.connector
import numpy as np

In [2]:
All_bus_info_to_sql = pd.read_csv("All_bus_info")

In [3]:
All_bus_info_to_sql.head(2)

,Bus Name,Bus Type,Start Time,End Time,Duration,Price,Seat Availability,Ratings,Route Name,Route Link,State,State_Name
0,NBSTC -SLG-ROH-DJL-01-7:00 - 161760,Non AC Seater 2+2,07:00,10:00,03h 00m,INR 105,29 Seats available,3.6\n254,Siliguri to Darjeeling,https://www.redbus.in/bus-tickets/siliguri-to-...,North Bengal,NaN
1,NBSTC -SLG-ROH-DJL-02-8:00 - 158470,Non AC Seater 2+2,08:00,11:00,03h 00m,INR 105,8 Seats available,4.0\n282,Siliguri to Darjeeling,https://www.redbus.in/bus-tickets/siliguri-to-...,North Bengal,NaN


In [4]:
##sql connection 
connection = mysql.connector.connect(
    host ="localhost",
    database = "Redbus_information",
    user="root",
    password="1234",
    port="3306"
)
cursor = connection.cursor()

In [5]:
#TABLE CREATION
create_query = '''CREATE TABLE IF NOT EXISTS REDBUS_TABLE(
    ID INT AUTO_INCREMENT PRIMARY KEY, 
    BUS_NAME VARCHAR(100), 
    BUS_TYPE TEXT, 
    START_TIME TIME, 
    END_TIME TIME, 
    DURATION VARCHAR(10), 
    PRICE DECIMAL(10, 2), 
    SEAT_AVAILABILITY INT, 
    RATINGS VARCHAR(10), 
    ROUTE_NAME VARCHAR(200),
    ROUTE_LINK TEXT,
    STATE TEXT
)'''

cursor.execute(create_query)
connection.commit()

In [17]:
#CLEANING DATA
# Replace NaN values with None
All_bus_info_to_sql = All_bus_info_to_sql.where(pd.notnull(All_bus_info_to_sql), None)

# Clean the 'Price' column: Remove 'INR ' and convert to float
All_bus_info_to_sql['Price'] = All_bus_info_to_sql['Price'].astype(str).str.replace('INR ', '').astype(float)

# Clean the 'Seat Availability' column: Replace non-digit characters and convert to integer
All_bus_info_to_sql['Seat Availability'] = All_bus_info_to_sql['Seat Availability'] \
    .replace('', '0') \
    .fillna('0') \
    .astype(str) \
    .str.replace(r'\D', '', regex=True) \
    .astype(int)




In [18]:
All_bus_info_to_sql.head()

,Bus Name,Bus Type,Start Time,End Time,Duration,Price,Seat Availability,Ratings,Route Name,Route Link,State
0,NBSTC -SLG-ROH-DJL-01-7:00 - 161760,Non AC Seater 2+2,07:00,10:00,03h 00m,105.0,29,3.6\n254,Siliguri to Darjeeling,https://www.redbus.in/bus-tickets/siliguri-to-...,North Bengal
1,NBSTC -SLG-ROH-DJL-02-8:00 - 158470,Non AC Seater 2+2,08:00,11:00,03h 00m,105.0,8,4.0\n282,Siliguri to Darjeeling,https://www.redbus.in/bus-tickets/siliguri-to-...,North Bengal
2,NORTH BENGAL STATE TRANSPORT CORPORATION - 171781,Non AC Seater (2+3),15:00,06:00,15h 00m,565.0,0,3.5\n25,Kolkata to Cooch Behar (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-c...,North Bengal
3,NBSTC-KOL-COB-R1-17:00 - 156429,Non AC Seater 2+2,17:00,08:00,15h 00m,608.0,0,3.3\n113,Kolkata to Cooch Behar (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-c...,North Bengal
4,NBSTC-BHP-COB-R1-18:30 - 156423,Non AC Seater 2+2,18:30,07:00,12h 30m,448.0,0,3.6\n71,Berhampore (West Bengal) to Cooch Behar (West ...,https://www.redbus.in/bus-tickets/berhampore-t...,North Bengal


In [19]:
#DATA INSERTION INTO TABLE 
insert_query ='''INSERT INTO REDBUS_TABLE(BUS_NAME ,BUS_TYPE ,START_TIME ,END_TIME ,DURATION ,PRICE ,SEAT_AVAILABILITY ,RATINGS ,
                                        ROUTE_NAME ,ROUTE_LINK ,STATE ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
data = All_bus_info_to_sql.values.tolist()
cursor.executemany(insert_query,data)
connection.commit() 

In [ ]:
data = All_bus_info_to_sql.values.tolist()
data[0]

In [16]:
All_bus_info_to_sql.head()

,Bus Name,Bus Type,Start Time,End Time,Duration,Price,Seat Availability,Ratings,Route Name,Route Link,State
0,NBSTC -SLG-ROH-DJL-01-7:00 - 161760,Non AC Seater 2+2,07:00,10:00,03h 00m,105.0,29,3.6\n254,Siliguri to Darjeeling,https://www.redbus.in/bus-tickets/siliguri-to-...,North Bengal
1,NBSTC -SLG-ROH-DJL-02-8:00 - 158470,Non AC Seater 2+2,08:00,11:00,03h 00m,105.0,8,4.0\n282,Siliguri to Darjeeling,https://www.redbus.in/bus-tickets/siliguri-to-...,North Bengal
2,NORTH BENGAL STATE TRANSPORT CORPORATION - 171781,Non AC Seater (2+3),15:00,06:00,15h 00m,565.0,0,3.5\n25,Kolkata to Cooch Behar (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-c...,North Bengal
3,NBSTC-KOL-COB-R1-17:00 - 156429,Non AC Seater 2+2,17:00,08:00,15h 00m,608.0,0,3.3\n113,Kolkata to Cooch Behar (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-c...,North Bengal
4,NBSTC-BHP-COB-R1-18:30 - 156423,Non AC Seater 2+2,18:30,07:00,12h 30m,448.0,0,3.6\n71,Berhampore (West Bengal) to Cooch Behar (West ...,https://www.redbus.in/bus-tickets/berhampore-t...,North Bengal
